In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Step 1: Load IMDb Dataset (Predefined)
# Load IMDb dataset using TensorFlow Datasets
(train_data, test_data), info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)

# Convert to lists for sklearn and TensorFlow
X_train, y_train = [], []
X_test, y_test = [], []

for text, label in train_data:
    X_train.append(text.numpy().decode('utf-8'))
    y_train.append(label.numpy())
for text, label in test_data:
    X_test.append(text.numpy().decode('utf-8'))
    y_test.append(label.numpy())

y_train = np.array(y_train)
y_test = np.array(y_test)

# Step 2: Sklearn Model (ML: Logistic Regression with TF-IDF)
# NLP: TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train Logistic Regression
lr_model = LogisticRegression(max_iter=200)
lr_model.fit(X_train_tfidf, y_train)

# Evaluate
y_pred_lr = lr_model.predict(X_test_tfidf)
lr_accuracy = accuracy_score(y_test, y_pred_lr)
print(f"Sklearn Logistic Regression Test Accuracy: {lr_accuracy:.4f}")

# Step 3: TensorFlow RNN Model (DL)
# NLP: Tokenization and Padding
vocab_size = 10000
max_len = 100
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

# Define RNN Model with LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=max_len),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate
test_loss, test_acc = model.evaluate(X_test_padded, y_test, verbose=0)
print(f"TensorFlow LSTM Test Accuracy: {test_acc:.4f}")

# Step 4: Predict on Custom Text (NLP: Inference)
def predict_sentiment(text, model, tokenizer, max_len):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')
    pred = model.predict(padded, verbose=0)[0][0]
    return "Positive" if pred > 0.5 else "Negative"

# Test predictions
test_texts = [
    "This movie was absolutely fantastic and thrilling!",
    "Terrible plot and boring acting."
]
for text in test_texts:
    print(f"'{text}' -> {predict_sentiment(text, model, tokenizer, max_len)}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\hp\tensorflow_datasets\imdb_reviews\plain_text\incomplete.SMS7Y7_1.0.0\imdb_reviews-train.t…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\hp\tensorflow_datasets\imdb_reviews\plain_text\incomplete.SMS7Y7_1.0.0\imdb_reviews-test.tf…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\hp\tensorflow_datasets\imdb_reviews\plain_text\incomplete.SMS7Y7_1.0.0\imdb_reviews-unsuper…

Dataset imdb_reviews downloaded and prepared to C:\Users\hp\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.
Sklearn Logistic Regression Test Accuracy: 0.8792
Epoch 1/5
625/625 [==============================] - 45s 65ms/step - loss: 0.5123 - accuracy: 0.7443 - val_loss: 0.4100 - val_accuracy: 0.8212
Epoch 2/5
625/625 [==============================] - 40s 64ms/step - loss: 0.3249 - accuracy: 0.8702 - val_loss: 0.4052 - val_accuracy: 0.8232
Epoch 3/5
625/625 [==============================] - 40s 63ms/step - loss: 0.2389 - accuracy: 0.9079 - val_loss: 0.4235 - val_accuracy: 0.8184
Epoch 4/5
625/625 [==============================] - 40s 64ms/step - loss: 0.1618 - accuracy: 0.9410 - val_loss: 0.5086 - val_accuracy: 0.8112
Epoch 5/5
625/625 [==============================] - 40s 64ms/step - loss: 0.1140 - accuracy: 0.9606 - val_loss: 0.5730 - val_accuracy: 0.8128
TensorFlow LSTM Test Accuracy: 0.7787
'This movie was absolutely fantastic and thrill